In [1]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras.utils import to_categorical
import cv2, numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import applications
from PIL import Image
Image.LOAD_TRUNCATED_IMAGES = True
import xml.etree.cElementTree as ET
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from shutil import copyfile
from pathlib import Path
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15468052222701746196
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11324823962
locality {
  bus_id: 1
}
incarnation: 18140930311599222436
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


# Test ResNet50

In [3]:
nbClasses = 5
img_rows, img_cols, img_channel = 224, 224, 3

base_model = applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
#add_model.add(Dense(256, activation='relu'))
#add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(nbClasses, activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [4]:
data_path = '/data/test_5classes/'

batch_size = 8
epochs = 8

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    data_path + 'Training/',  
    target_size=(224,224),  
    batch_size=batch_size)  


validation_generator = validation_datagen.flow_from_directory(
        data_path + 'Validation/',
        target_size=(224,224),
        batch_size=batch_size)

test_generator = test_datagen.flow_from_directory(
        data_path + 'Test/',
        target_size=(224,224),
        batch_size=batch_size)

model.fit_generator(
    train_generator,
    steps_per_epoch= 299 // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[ModelCheckpoint(data_path + 'VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]
)
#model.save_weights(data_path + 'weights/try_VGG16.h5')

model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.0001, momentum=0.9),
              metrics=['accuracy'])
              
model.fit_generator(
    train_generator,
    steps_per_epoch= 299 // batch_size,
    epochs=2,
    validation_data=validation_generator,
    callbacks=[ModelCheckpoint(data_path + 'VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]
)

Found 299 images belonging to 5 classes.
Found 129 images belonging to 5 classes.
Found 109 images belonging to 5 classes.
Epoch 1/8
37/37 [==============================] - 10s 272ms/step - loss: 1.5403 - acc: 0.3840 - val_loss: 1.5757 - val_acc: 0.3488
Epoch 2/8
37/37 [==============================] - 6s 173ms/step - loss: 1.2486 - acc: 0.4922 - val_loss: 1.7832 - val_acc: 0.2868
Epoch 3/8
37/37 [==============================] - 6s 173ms/step - loss: 1.0777 - acc: 0.5632 - val_loss: 2.0192 - val_acc: 0.2868
Epoch 4/8
37/37 [==============================] - 6s 171ms/step - loss: 0.9928 - acc: 0.5801 - val_loss: 2.3349 - val_acc: 0.2868
Epoch 5/8
37/37 [==============================] - 6s 167ms/step - loss: 0.9665 - acc: 0.6037 - val_loss: 3.0280 - val_acc: 0.2868
Epoch 6/8
37/37 [==============================] - 6s 174ms/step - loss: 0.8701 - acc: 0.6531 - val_loss: 3.7863 - val_acc: 0.2868
Epoch 7/8
37/37 [==============================] - 6s 175ms/step - loss: 0.8433 - acc: 0.6

In [6]:
print(predictions)

NameError: name 'predictions' is not defined